# RAPIDS - Genetic Algorithm KNN - [CV 0.01840]
The RAPIDS library is now available in all Kaggle notebooks. Hooray! Simply type `import cuml` or `import cudf` to load the two most popular packages.

RAPIDS is described [here][1]. RAPIDS `cuDF` accelerates dataframe operations using GPU and has a similar api as Pandas. RAPIDS `cuML` accelerates machine learning algorithms using GPU and has a similar api as Scikit-Learn. Since RAPIDS ML algorithms are so fast, we can do things that were never possible like applying genetic algorithms to ML hyperparameter searchs!

The ML algorithm kNN is very sensitive to the scaling of the feature columns. In this notebook, we find the weights for feature columns (`0 <= w <= 1`) using a genetic algorithm. This is similar to a feature selection algorithm like ridge regression which shrinks the importance of features. Alternatively we could mimic lasso regression if we choose only weights `w = 0` and `w = 1`.

[1]: https://rapids.ai/

# Load Libraries

In [ ]:
import sys, warnings
warnings.filterwarnings("ignore")
sys.path.append('../input/iterativestratification')

import pandas as pd, numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import log_loss

import cuml
print('RAPIDS',cuml.__version__)

# Load Data

In [ ]:
train = pd.read_csv('../input/lish-moa/train_features.csv')
print('train shape',train.shape)
test = pd.read_csv('../input/lish-moa/test_features.csv')
print('test shape',test.shape)
targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')

In [ ]:
train.cp_dose = train.cp_dose.map({'D1':-1,'D2':1})
test.cp_dose = test.cp_dose.map({'D1':-1,'D2':1})

train.cp_time = train.cp_time.map({24:-1, 48:0, 72:1})
test.cp_time = test.cp_time.map({24:-1, 48:0, 72:1})

train.cp_type = train.cp_type.map({'trt_cp':-1, 'ctl_vehicle':1})
test.cp_type = test.cp_type.map({'trt_cp':-1, 'ctl_vehicle':1})

train.head()

# Double Stratified K Fold
We will use double stratified folds discussed [here][1]
  
[1]: https://www.kaggle.com/c/lish-moa/discussion/195195

In [ ]:
def make_folds(folds = 5, random_state = 0, stratify = True, scored = None):
    
    drug = pd.read_csv('../input/lish-moa/train_drug.csv')
    if scored is None:
        scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
    targets = scored.columns[1:]
    scored = scored.merge(drug, on='sig_id', how='left')

    # LOCATE DRUGS
    vc = scored.drug_id.value_counts()
    vc1 = vc.loc[vc<=18].index.sort_values()
    vc2 = vc.loc[vc>18].index.sort_values()

    # STRATIFY DRUGS 18 OR LESS
    dct1 = {}; dct2 = {}
    if stratify:
        skf = MultilabelStratifiedKFold(n_splits=folds, shuffle=True, random_state=random_state)
    else:
        skf = KFold(n_splits=folds, shuffle=True, random_state=random_state)
    tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.index[idxV].values}
        dct1.update(dd)
    
    # STRATIFY DRUGS MORE THAN 18
    if stratify:
        skf = MultilabelStratifiedKFold(n_splits=folds, shuffle=True, random_state=random_state)
    else:
        skf = KFold(n_splits=folds, shuffle=True, random_state=random_state)
    tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.sig_id[idxV].values}
        dct2.update(dd)
    
    # ASSIGN FOLDS
    scored['fold'] = np.nan
    scored['fold'] = scored.drug_id.map(dct1)
    scored.loc[scored.fold.isna(),'fold'] = scored.loc[scored.fold.isna(),'sig_id'].map(dct2)
    scored.fold = scored.fold.astype('int8')
    
    return scored[['sig_id','fold']].copy()

# Initialize Variables
Our genetic algorithm works as follows. Each generation, we take the best `sqrt(POPULATION)` sets of weights and then we randomly combine them in pairs to create a new population of size `POPULATION`. We also apply `MUTATE` percentage of mutations (i.e. randomly change some weights). We can think of a set of weights as "DNA".

In [ ]:
GENERATIONS = 20
POPULATION = 100 # must be perfect square
PARENTS = int( np.sqrt(POPULATION) )
MUTATE = 0.05

# RANDOMLY CREATE CV
FOLDS = 5; SEED = 42
ff = make_folds(folds=FOLDS, random_state=SEED, stratify=True, scored=targets)
train['fold'] = ff.fold.values
targets['fold'] = ff.fold.values

# INITIALIZE
oof = np.zeros((len(train),206))
dna = np.random.uniform(0,1,(POPULATION,875))**2.0
cvs = np.zeros((POPULATION))

# Genetic Search

In [ ]:
%%time
for jj in range(GENERATIONS):

    # ALL PREVIOUS POPULATION "DNA" AND CV SCORES
    df = pd.DataFrame(data=dna)
    df['cv'] = cvs
    df.sort_values('cv',inplace=True)
    print('Evolving...')

    # GENERATE AND EVALUATE CHILDREN
    for k in range(POPULATION):
        print(k,', ',end='')
        
        # GENERATE CHILD
        if jj!=0:
            parent1 = k//PARENTS; parent2 = k%PARENTS
            TMP = np.random.randint(0,2,875)
            dna[k,] = TMP * df.iloc[parent1,:-1] + (1-TMP) * df.iloc[parent2,:-1]
            x = np.random.uniform(0,1,875)
            IDX = np.where(x<MUTATE)[0]
            dna[k,IDX] = np.random.uniform(0,1,len(IDX))**2.0
        else:
            dna[k,] = df.iloc[k,:-1]
        
        # KNN WEIGHTS
        WGT = dna[k,]
        # WEIGHTS FOR CP_TYPE, CP_TIME, CP_DOSE
        WGT[0]= 100 ; WGT[1] = 12/2; WGT[2] = 5/2

        # KNN KFOLD VALIDATE
        for fold in range(FOLDS):    
            model = cuml.neighbors.KNeighborsClassifier(n_neighbors=1000)
            model.fit( train.loc[ train.fold!=fold, train.columns[1:-1] ].values * WGT,
                       targets.loc[targets.fold!=fold, targets.columns[1:-1] ] )
    
            pp = model.predict_proba( train.loc[ train.fold==fold, train.columns[1:-1] ].values * WGT )
            pp = np.stack( [(1 - pp[x][:,0]) for x in range(len(pp))] ).T
            oof[targets.fold==fold,] = pp
        
        cv_score = log_loss( targets.iloc[:,1:-1].values.flatten(), oof.flatten() )
        cvs[k] = cv_score
        
    # DISPLAY BEST "DNA"
    RES = 10
    idx = np.argmin(cvs)
    plt.figure(figsize=(20,3))
    plt.plot(np.arange(872)[::RES],dna[idx,3:][::RES],'o-')
    plt.title('Best "DNA" after generation %i yields CV SCORE = %.5f'%(jj+1,np.min(cvs)),size=16)
    plt.show()    

# Show all population "DNA" with CV scores
Each row in the dataframe below is one set of weights from our evolved population. The first column indicates the cv score and the next 875 columns are the weights that we scale the 875 features with. (The weights can be thought of as "DNA" in the context of genetic algorithm). We will save the "DNA" to disk.

In [ ]:
df = pd.DataFrame(data=dna)
df['cv'] = cvs
df = df[['cv']+list(np.arange(0,875))]
df.sort_values('cv',inplace=True)
df.to_csv('dna.csv',index=False)
df.head()

# Predict Test Data

In [ ]:
WGT = df.iloc[0,1:].values
oof = np.zeros((len(train),206))
preds = np.zeros((len(test),206))

for fold in range(FOLDS):
    print('FOLD %i'%(fold+1), ' ', end='')
    
    model = cuml.neighbors.KNeighborsClassifier(n_neighbors=1000)
    model.fit( train.loc[ train.fold!=fold, train.columns[1:-1] ].values * WGT,
               targets.loc[targets.fold!=fold, targets.columns[1:-1] ] )
    
    pp = model.predict_proba( train.loc[ train.fold==fold, train.columns[1:-1] ].values * WGT )
    pp = np.stack( [(1 - pp[x][:,0]) for x in range(len(pp))] ).T
    oof[targets.fold==fold,] = pp
    
    pp = model.predict_proba( test[test.columns[1:]].values * WGT )
    pp = np.stack( [(1 - pp[x][:,0]) for x in range(len(pp))] ).T
    preds += pp/FOLDS    
    
print()
cv_score = log_loss( targets.iloc[:,1:-1].values.flatten(), oof.flatten() )
print('CV SCORE = %.5f'%cv_score)

# Clip and Submit
If we predict 0 when the true target is 1, we are severly penalized in this competition's metric. Therefore we clip our predictions and observe an increase of CV LB 0.00050! We will save the OOF to disk before clipping.

In [ ]:
df_oof = targets.copy()
df_oof.iloc[:,1:-1] = oof
df_oof.to_csv('oof.csv',index=False)

oof = np.clip(oof,0.0005,0.999)
oof[train.cp_type==1,] = 0

cv_score = log_loss( targets.iloc[:,1:-1].values.flatten(), oof.flatten() )
print('CV SCORE with CLIPPING = %.5f'%cv_score)

In [ ]:
sub = pd.read_csv('../input/lish-moa/sample_submission.csv')
sub.iloc[:,1:] = np.clip(preds,0.0005,0.999)
sub.loc[test.cp_type==1,sub.columns[1:]] = 0
sub.to_csv('submission.csv',index=False)
sub.head()

In [ ]:
targets.iloc[:,1:-1].mean()

In [ ]:
sub.iloc[:,1:].mean()